In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import numpy as np
from numba import njit
import numba
from nonlinshrink import nonlinshrink as nls
from nonlinshrink import nonlinshrink_numba as nls_numba


def test_without_numba():
    np.random.seed(2022)
    ps = np.arange(start=10, stop=200, step=20)

    e_prial = np.zeros(len(ps))
    for i, p in enumerate(ps):
        reps = int(np.maximum(100, np.minimum(100, 1e5 / p)))
        prial = np.zeros(reps, dtype=float)
        for j in np.arange(reps):

            n = 600
            lam = np.concatenate(
                [
                    np.ones(int(p / 5)),
                    3 * np.ones(int(2 * p / 5.0)),
                    10 * np.ones(int(2 * p / 5.0)),
                ]
            )
            sigma = np.diag(np.ones(p) * lam)
            xx = np.random.randn(n, p)
            d, u = np.linalg.eigh(sigma)
            # y = np.linalg.solve(u.T, xx.T)
            s_sqrt = np.eye(p, p) * np.sqrt(lam)
            y = np.dot(xx, s_sqrt)
            s_tilde = nls.shrink_cov(y)
            s_sample = np.cov(y.T)
            pr = nls.prial(s_sample, s_tilde, sigma)

            prial[j] = float(pr)

        e_prial[i] = np.mean(prial)
    return e_prial

@njit
def test_with_numba():
    np.random.seed(2022)
    ps = np.arange(start=10, stop=200, step=20)

    e_prial = np.zeros(len(ps))
    for i, p in enumerate(ps):
        reps = int(np.maximum(100, np.minimum(100, 1e5 / p)))
        prial = np.zeros(reps)
        for j in np.arange(reps):

            n = 600
            lam = np.concatenate(
                (
                    np.ones(int(p / 5)),
                    3 * np.ones(int(2 * p / 5.0)),
                    10 * np.ones(int(2 * p / 5.0)),
                )
            )
            sigma = np.diag(np.ones(p) * lam)
            xx = np.random.randn(n, p)
            d, u = np.linalg.eigh(sigma)
            # y = np.linalg.solve(u.T, xx.T)
            s_sqrt = np.eye(p, p) * np.sqrt(lam)
            y = np.dot(xx, s_sqrt)
            s_tilde = nls_numba.shrink_cov(y)
            s_sample = np.cov(y.T)
            pr = nls_numba.prial(s_sample, s_tilde, sigma)

            prial[j] = float(pr)

        e_prial[i] = np.mean(prial)
    return e_prial

In [9]:
test_cov_mat = np.random.rand(20,20)
test_cov_mat = np.dot(test_cov_mat, test_cov_mat.T)

In [10]:
%%timeit
nls_numba.shrink_cov(test_cov_mat)

57.8 µs ± 25.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
nls.shrink_cov(test_cov_mat) 

225 µs ± 2.07 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
